In [ ]:
import pandas as pd, numpy as np
from IPython.display import display
from IPython.display import Image
import math

# Sélection des données

In [ ]:
df = pd.read_csv('ALL_VAR_30_POP.csv')
df.info()

In [ ]:
df.head()

In [ ]:
#sélection des pays en voie de développement
countries_df = pd.read_csv('developping_countries.csv')
countries=countries_df['area']
df=df[df.area.isin(countries)]

In [ ]:
#sélection du type de cancer
df=df[df.type == 'C16']
df=df.drop('type', axis=1)

In [ ]:
df.info()

# Choix du modèle

In [ ]:
model="knn"

# Split des données

In [ ]:
import sklearn
import sklearn.cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [ ]:
#variable à prédire : mortalité relative
X=df.drop('relative_mortality', axis=1)
Y=df.relative_mortality

In [ ]:
#standardisation des variables d'entrée pour les modèles linéaires
if model in ["linear_regression", "ridge_regression", "lasso_regression"]:
    labels = X.columns
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(scaler.fit_transform(X), columns=labels)

In [ ]:
#split
X_train, X_test, Y_train, Y_test = sklearn.cross_validation.train_test_split(X, Y, test_size=0.33, random_state = 5)
print("Dimensions :")
print("X_train : {}".format(X_train.shape))
print("X_test : {}".format(X_test.shape))
print("Y_train : {}".format(Y_train.shape))
print("Y_test : {}".format(Y_test.shape))

In [ ]:
#variables population totale, année et pays mises de coté
X_results=X_test['area', 'year', 'TOTAL_POP']

X_test=X_test.drop('TOTAL_POP', axis=1)
X_train=X_train.drop('TOTAL_POP', axis=1)

X_test=X_test.drop('area', axis=1)
X_train=X_test.drop('area', axis=1)

X_test=X_test.drop('year', axis=1)
X_train=X_test.drop('year', axis=1)

# Validation croisée pour déterminer les hyperparamètres

In [ ]:
import matplotlib.pyplot as plt
import csv

from sklearn.linear_model import LinearRegression
from sklearn import linear_model

from sklearn.neighbors import KNeighborsRegressor

from sklearn import tree

## Régression linéaire

In [ ]:
if model=="linear_regression":
    #pas d'hyperparamètre à déterminer

    #apprentissage du modèle
    model = linear_model.LinearRegression()
    model.fit(X_train, Y_train)
    print("Coefs : {}".format(list(zip(X_train.columns.values, model.coef_))))

## Régression linéaire ridge

In [ ]:
if model=="ridge_regression":
    
    alphas = []
    train_scores = []
    scores = []
    intercepts = []

    for i in range(10):
        alpha = 0.01*(i)
        alphas.append(alpha)
        reg = linear_model.Ridge(alpha = alpha)
        reg.fit(x_train, y_train)
        train_scores.append(reg.score(x_train, y_train))
        print("Score with Ridge with train, alpha = {} : {}".format(alpha, reg.score(x_train, y_train)))
        print("Score with Ridge, alpha = {} : {}".format(alpha, reg.score(x_test, y_test)))
        scores.append(reg.score(x_test, y_test))
        #print("Intercept : {}".format(reg.intercept_))
        intercepts.append(reg.intercept_)
        #print("Coefs : {}".format(list(zip(x_train.columns.values, reg.coef_))))

    for i in range(5):
        alpha = 0.1*(2*i)
        alphas.append(alpha)
        reg = linear_model.Ridge(alpha = alpha)
        reg.fit(x_train, y_train)
        train_scores.append(reg.score(x_train, y_train))
        print("Score with Ridge with train, alpha = {} : {}".format(alpha, reg.score(x_train, y_train)))
        print("Score with Ridge, alpha = {} : {}".format(alpha, reg.score(x_test, y_test)))
        scores.append(reg.score(x_test, y_test))
        #print("Intercept : {}".format(reg.intercept_))
        intercepts.append(reg.intercept_)
        #print("Coefs : {}".format(list(zip(x_train.columns.values, reg.coef_))))

    for i in range(10):
        alpha = (2*i+1)
        alphas.append(alpha)
        reg = linear_model.Ridge(alpha = alpha)
        reg.fit(x_train, y_train)
        train_scores.append(reg.score(x_train, y_train))
        print("Score with Ridge with train, alpha = {} : {}".format(alpha, reg.score(x_train, y_train)))
        print("Score with Ridge, alpha = {} : {}".format(alpha, reg.score(x_test, y_test)))
        scores.append(reg.score(x_test, y_test))
        #print("Intercept : {}".format(reg.intercept_))
        intercepts.append(reg.intercept_)
        #print("Coefs : {}".format(list(zip(x_train.columns.values, reg.coef_))))

    with open('ridge.csv', 'wt') as ridge_csv:
        writer = csv.writer(ridge_csv, delimiter = ';')
        writer.writerow(alphas)
        writer.writerow(train_scores)
        writer.writerow(scores)
        writer.writerow(intercepts)
    
    #apprentissage du modèle#############utiliser le bon alpha
    model = linear_model.Ridge()
    model.fit(X_train, Y_train)

## Régression linéaire lasso

In [ ]:
if model=="lasso_regression"

    alphas = []
    train_scores = []
    scores = []
    intercepts = []
    non_zero_number = []
    non_zero_values = []
    non_zero_coefs = []

    for i in range(10):
        alpha = 0.001*(i+1)
        alphas.append(alpha)

        reg = linear_model.Lasso(alpha = alpha)
        reg.fit(x_train, y_train)
        train_scores.append(reg.score(x_train, y_train))
        print("Score with Lasso with train, alpha = {} : {}".format(alpha, reg.score(x_train, y_train)))
        print("Score with Lasso, alpha = {} : {}".format(alpha, reg.score(x_test, y_test)))
        scores.append(reg.score(x_test, y_test))
        #print("Intercept : {}".format(reg.intercept_))
        intercepts.append(reg.intercept_)
        #print("Coefs : {}".format(list(zip(x_train.columns.values, reg.coef_))))
        count = 0
        non_zero_list = []
        non_zero_coefs_list = []
        for i, val in enumerate(reg.coef_):
            if val !=0:
                #print("Non-Zero Coef {} for alpha = {} : {}".format(x_train.columns.values[i], alpha, reg.coef_[i]))
                count+=1
                non_zero_list.append(x_train.columns.values[i])
                non_zero_coefs_list.append(str(reg.coef_[i]))
        non_zero_number.append(count)
        non_zero_coefs.append(non_zero_coefs_list)
        non_zero_values.append(non_zero_list)
        #print("Number of non-zero coefficients with Lasso, alpha = {} : {}".format(alpha, count))


    for i in range(10):
        alpha = 0.01*(i+1)
        alphas.append(alpha)

        reg = linear_model.Lasso(alpha = alpha)
        reg.fit(x_train, y_train)
        train_scores.append(reg.score(x_train, y_train))
        print("Score with Lasso with train, alpha = {} : {}".format(alpha, reg.score(x_train, y_train)))
        print("Score with Lasso, alpha = {} : {}".format(alpha, reg.score(x_test, y_test)))
        scores.append(reg.score(x_test, y_test))
        #print("Intercept : {}".format(reg.intercept_))
        intercepts.append(reg.intercept_)
        #print("Coefs : {}".format(list(zip(x_train.columns.values, reg.coef_))))
        count = 0
        non_zero_list = []
        non_zero_coefs_list = []
        for i, val in enumerate(reg.coef_):
            if val !=0:
                #print("Non-Zero Coef {} for alpha = {} : {}".format(x_train.columns.values[i], alpha, reg.coef_[i]))
                count+=1
                non_zero_list.append(x_train.columns.values[i])
                non_zero_coefs_list.append(str(reg.coef_[i]))
        non_zero_number.append(count)
        non_zero_coefs.append(non_zero_coefs_list)
        non_zero_values.append(non_zero_list)
        #print("Number of non-zero coefficients with Lasso, alpha = {} : {}".format(alpha, count))

    for i, alpha in enumerate(alphas):
        decimal_split = '.'.split(str(alpha))
        ext = ''
        if len(decimal_split)>1:
            ext = decimal_split[1]
        else:
            ext = str(alpha)
        with open('./lasso/lasso_{}.csv'.format(ext), 'wt') as file:
            csvwrite = csv.writer(file, delimiter = ';')
            for j in range(non_zero_number[i]):
                csvwrite.writerow([non_zero_values[i][j], non_zero_coefs[i][j]])

    with open('lasso.csv', 'wt') as lasso_csv:
        writer = csv.writer(lasso_csv, delimiter = ';')
        writer.writerow(alphas)
        writer.writerow(train_scores)
        writer.writerow(scores)
        writer.writerow(intercepts)
        writer.writerow(non_zero_number)
    
    #apprentissage du modèle######################utiliser le bon alpha
    model = linear_model.Lasso()
    model.fit(X_train, Y_train)

## K plus proches voisins

In [ ]:
if model=="knn":
    
    #hyperparamètre : nombre k de plus proches voisins
    for k in range(1, 10):
    print("{} voisins".format(k))
    neigh = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(neigh, X_train, Y_train, cv=5)
    print("Uniform weights : accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    neigh = KNeighborsRegressor(n_neighbors=k, weights='distance')
    scores = cross_val_score(neigh, X_train, Y_train, cv=5)
    print("Distance weights : accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    #apprentissage du modèle
    model = KNeighborsRegressor(n_neighbors=3, weights='distance')
    model.fit(X_train, Y_train)

## Arbres de décision

In [ ]:
if model=="decision_tree":
    model=tree.DecisionTreeRegressor()
    model.fit(X_train, Y_train)
    
    score = model.score(X_test, Y_test)
    print(score)
    
    for i in range(len(tr.feature_importances_)):
        print("{}".format(str(list(X.columns.values)[i])))
        print("{}".format(str(tr.feature_importances_[i])))

## Random forest

# Evaluation du modèle

In [ ]:
#coefficient de détermination
model.score(X_test, Y_test)

In [ ]:
#prédiction de la mortalité en volume à partir de la mortalité relative prédite par le modèle
Y_predicted=model.predict(X_test)
X_results['predicted_relative_mortality'] = Y_predicted
X_results['relative_mortality']=Y_test
X_results['predicted_mortality']=X_results['predicted_relative_mortality']*X_results['TOT_POP']
X_results['predicted_mortality']=X_results['predicted_mortality'].round()
X_results['mortality']=X_results['relative_mortality']*X_results['TOT_POP']

In [ ]:
#Mean Square Error
mse_test = np.mean((X_results['mortality'] - X_results['predicted_mortality']) **2)
mse_test

In [ ]:
#Root Mean Square Error
rmse_test=math.sqrt(mse_test)
rmse_test

In [ ]:
#Mean Average Error
mae_test=np.mean(abs(X_test['mortality'] - X_test['predicted_mortaltity']))
mae_test

In [ ]:
#Relative Mean Average Error
mean_mortality_test=np.mean(X_test['mortality'])
rel_mae=mae_test/mean_mortality_test
rel_mae

developping countries
standard scaling sur variables d'entrée uniquement ?
cross val sur ridge, lasso, arbres de décision
script spécial PCA ?